# Clase Práctica #7 (Compilación)

En esta clase estaremos implementando algunas de las operaciones más importantes entre autómatas. Específicamente implementaremos **unión**, **concatenación** y **clausura**, las cuales juegan un papel fundamental en la construcción del intérprete de expresiones regulares en el que trabajaremos la semana próxima.

In [ ]:
from cmp.tools.automata import NFA, DFA, nfa_to_dfa

## Operaciones entre autómatas

Trabajemos con el siguiente autómata de ejemplo. El lenguaje que reconoce dicho autómata es `(a|b)*b`.

In [ ]:
automaton = DFA(states=2, finals=[1], transitions={
    (0,'a'):  0,
    (0,'b'):  1,
    (1,'a'):  0,
    (1,'b'):  1,
})
automaton

### Unión de autómatas

Dados 2 autómatas finitos `a1` y `a2` se define el autómata **unión** entre ellos como el autómata que reconoce $L(a_1) \cup L(a_2)$. Dicho de otra forma, reconoce el lenguaje $L(a_1 \cup a_2) = \{w $| $w \in L(a_1) \lor w \in L(a_2)\}$.

In [ ]:
def automata_union(a1, a2):
    transitions = {}
    
    start = 0
    d1 = 1
    d2 = a1.states + d1
    final = a2.states + d2
    
    for (origin, symbol), destinations in a1.map.items():
        ## Relocate a1 transitions ...
        # Your code here
        pass

    for (origin, symbol), destinations in a2.map.items():
        ## Relocate a2 transitions ...
        # Your code here
        pass
    
    ## Add transitions from start state ...
    # Your code here
    
    ## Add transitions to final state ...
    # Your code here
            
    states = a1.states + a2.states + 2
    finals = { final }
    
    return NFA(states, finals, transitions, start)

Comprobemos que `automata_union(automaton, automaton)` reconoce el propio lenguaje `(a|b)*b`.

In [ ]:
union = automata_union(automaton, automaton)
display(union)
recognize = nfa_to_dfa(union).recognize

assert union.states == 2 * automaton.states + 2

assert recognize('b')
assert recognize('abbb')
assert recognize('abaaababab')

assert not recognize('')
assert not recognize('a')
assert not recognize('abbbbaa')

### Concatenación de autómatas

Dados 2 autómatas finitos `a1` y `a2` se define el autómata **concatenación** entre ellos como el autómata que reconoce $L(a_1) L(a_2)$. Dicho de otra forma, reconoce el lenguaje $L(a_1 a_2) = \{vw $| $v \in L(a_1) \land w \in L(a_2)\}$.

In [ ]:
def automata_concatenation(a1, a2):
    transitions = {}
    
    start = 0
    d1 = 0
    d2 = a1.states + d1
    final = a2.states + d2
    
    for (origin, symbol), destinations in a1.map.items():
        ## Relocate a1 transitions ...
        # Your code here
        pass

    for (origin, symbol), destinations in a2.map.items():
        ## Relocate a2 transitions ...
        # Your code here
        pass
    
    ## Add transitions to final state ...
    # Your code here
            
    states = a1.states + a2.states + 1
    finals = { final }
    
    return NFA(states, finals, transitions, start)

Comprobemos que `automata_concatenation(automaton, automaton)` reconoce el lenguaje `(a|b)*b(a|b)*b`.

In [ ]:
concat = automata_concatenation(automaton, automaton)
display(concat)
recognize = nfa_to_dfa(concat).recognize

assert concat.states == 2 * automaton.states + 1

assert recognize('bb')
assert recognize('abbb')
assert recognize('abaaababab')

assert not recognize('')
assert not recognize('a')
assert not recognize('b')
assert not recognize('ab')
assert not recognize('aaaab')
assert not recognize('abbbbaa')

### Clausura de autómatas

Dado un autómata finito `a1` se define el autómata **clausura** de `a1` como el autómata que reconoce $L(a_1)^*$. Dicho de otra forma, reconoce el lenguaje $L(a_1^*) = L(a_1)^n$, con $n \geq 0$.

In [ ]:
def automata_closure(a1):
    transitions = {}
    
    start = 0
    d1 = 1
    final = a1.states + d1
    
    for (origin, symbol), destinations in a1.map.items():
        ## Relocate automaton transitions ...
        # Your code here
    
    ## Add transitions from start state ...
    # Your code here
    
    ## Add transitions to final state and to start state ...
    # Your code here
            
    states = a1.states +  2
    finals = { final }
    
    return NFA(states, finals, transitions, start)

Comprobemos que `automata_closure(automaton)` reconoce el lenguaje `((a|b)*b)*`.

In [ ]:
closure = automata_closure(automaton)
display(closure)
recognize = nfa_to_dfa(closure).recognize

assert closure.states == automaton.states + 2

assert recognize('')
assert recognize('b')
assert recognize('ab')
assert recognize('bb')
assert recognize('abbb')
assert recognize('abaaababab')

assert not recognize('a')
assert not recognize('abbbbaa')

### Minimización de autómatas

Hasta ahora hemos estado ignorando la cantidad de estados del autómata. Sin embargo, resulta conveniente obtener el DFA con menor cantidad de estados. Se puede demostrar que siempre hay un único DFA con la mínima cantidad de estados para cualquier lenguaje regular. Dicho autómata puede ser construido a partir de cualquier otro DFA del lenguaje como resultado de agrupar conjuntos de estados equivalentes.

El algoritmo de minimización particiona los estados del DFA en grupos de estados que no puedan ser diferenciados _(comenzando en ellos, cualquier cadena termina siendo igualmente aceptada o no)_. Cada grupo de estados es mezclado posteriormente en un único estado del DFA resultante. A medida que avanza el algoritmo, los estados en el mismo grupo representan estados que aún no se han podido diferenciar, mientras que dos estados cualesquiera de distintos grupos ya fueron diferenciados. Una vez que la partición no pueda ser refinada más, el algoritmo se detiene pues encontró el DFA con menor cantidad de estados.

El algoritmo funciona de la siguiente forma:
1. Comenzar con una partición inicial $\Pi$ en dos grupos, $F$ y $Q - F$, con los estados finales y no finales respectivamente.
2. Inicializar $\Pi_{new} = \Pi$
3. Por cada grupo $G$ en la partición $\Pi$:
    1. Particionar $G$ en subgrupos de forma tal que: dos estados $s$ y $t$ están en el mismo grupo **si y solo si** para todo símbolo $a$ del vocabulario, los estados $s$ y $t$ tienen transición con $a$ hacia estados en el mismo grupo de $\Pi$.
    2. Reemplazar $G$ in $\Pi_{new}$ por los subgrupos formados.
4. Si $\Pi_{new} = \Pi$, hacer $\Pi_{final} = \Pi$ y continuar al paso `5`. En otro caso, regresar al paso `2` con $\Pi_{new}$ en el lugar de $\Pi$.
5. Seleccionar un estado de cada grupo de $\Pi_{final}$ como el _representante_ de ese grupo. Los representantes serán los estados del DFA con la mínima cantidad de estados. El resto de los componentes se construyen de la siguiente forma:
    1. El estado inicial es el representante del grupo que contiene al estado inicial del autómata original.
    2. Los estados finales son los representantes de los grupos que contienen uno de los estados finales originales. _Note que cada grupo contiene solamente estados finales o no finales, ya que se parte de una partición que los separa y nunca se vuelven a unir_.
    3. Las transiciones se obtienen entre los representantes de cada grupo. Sea $s$ el representante de algún grupo de $G$, y sea, $(s, a) \to t$, la transición del autómata original desde $s$ con un símbolo $a$. Sea $r$ el representante de $t$ en su grupo $H$. Entonces en el autómata resultante estará la transición: $(s, a) \to r$.

Utilizaremos la implementación de la estructura de datos _conjuntos disjuntos_ de `cmp.utils`. A continuación se ilustra el uso de las funcionalidades más relevantes de la mismas.

In [ ]:
from cmp.utils import DisjointSet

dset = DisjointSet(*range(10))
print('> Inicializando conjuntos disjuntos:\n', dset)

dset.merge([5,9])
print('> Mezclando conjuntos 5 y 9:\n', dset)

dset.merge([8,0,2])
print('> Mezclando conjuntos 8, 0 y 2:\n', dset)

dset.merge([2,9])
print('> Mezclando conjuntos 2 y 9:\n', dset)

print('> Representantes:\n', dset.representatives)
print('> Grupos:\n', dset.groups)
print('> Nodos:\n', dset.nodes)
print('> Conjunto 0:\n', dset[0], '--->', type(dset[0]))
print('> Conjunto 0 [valor]:\n', dset[0].value, '--->' , type(dset[0].value))
print('> Conjunto 0 [representante]:\n', dset[0].representative, '--->' , type(dset[0].representative))

Trabajaremos sobre el siguiente autómamata que reconoce el lenguaje `(a|b)*abb`. Note que los estados `0` y `2` pueden ser mezclados.

In [ ]:
automaton = DFA(states=5, finals=[4], transitions={
    (0,'a'): 1,
    (0,'b'): 2,
    (1,'a'): 1,
    (1,'b'): 3,
    (2,'a'): 1,
    (2,'b'): 2,
    (3,'a'): 1,
    (3,'b'): 4,
    (4,'a'): 1,
    (4,'b'): 2,
})
automaton

Implementemos los métodos siguientes, donde:

- `distinguish_states(group, automaton, partition)`: devuelve los subgrupos que se obtienen de diferenciar los estados de `group` según la partición `partition`.
> `int[][]`: Lista de las listas de estados en las que se dividió `group`.

- `state_minimization(automaton)`: devuelve una partición de los estados de `automaton`.
> `DisjointSet`: Conjunto disjunto sobre los estados de `automaton`.

- `automata_minimization(automaton)`: devuelve el DFA con la mínima cantidad de estados que reconoce el mismo lenguaje que el DFA `automaton`.

In [ ]:
def distinguish_states(group, automaton, partition):
    split = {}
    vocabulary = tuple(automaton.vocabulary)

    for member in group:
        # Your code here
        pass

    return [ group for group in split.values()]
            
def state_minimization(automaton):
    partition = DisjointSet(*range(automaton.states))
    
    ## partition = { NON-FINALS | FINALS }
    # Your code here
    
    while True:
        new_partition = DisjointSet(*range(automaton.states))
        
        ## Split each group if needed (use distinguish_states(group, automaton, partition))
        # Your code here

        if len(new_partition) == len(partition):
            break

        partition = new_partition
        
    return partition

def automata_minimization(automaton):
    partition = state_minimization(automaton)
    
    states = [s for s in partition.representatives]
    
    transitions = {}
    for i, state in enumerate(states):
        ## origin = ???
        # Your code here
        for symbol, destinations in automaton.transitions[origin].items():
            # Your code here
            
            try:
                transitions[i,symbol]
                assert False
            except KeyError:
                # Your code here
                pass
    
    ## finals = ???
    ## start  = ???
    # Your code here
    
    return DFA(len(states), finals, transitions, start)

Comprobemos que al minimizar los estados del autómata, solo los estados `0` y `2` quedan en el mismo grupo, pues no son diferenciables.

In [ ]:
states = state_minimization(automaton)
print(states)

for members in states.groups:
    all_in_finals = all(m.value in automaton.finals for m in members)
    none_in_finals = all(m.value not in automaton.finals for m in members)
    assert all_in_finals or none_in_finals
    
assert len(states) == 4
assert states[0].representative == states[2].representative
assert states[1].representative == states[1]
assert states[3].representative == states[3]
assert states[4].representative == states[4]

Comprobemos que el algoritmo de minimización funciona correctamente. La cantidad de estados debe haberse reducido puesto que los estados `0` y `2` no son diferenciables. El autómata debe seguir reconociendo el lenguaje `(a|b)*abb`.

In [ ]:
mini = automata_minimization(automaton)
display(mini)

assert mini.states == 4

assert mini.recognize('abb')
assert mini.recognize('ababbaabb')

assert not mini.recognize('')
assert not mini.recognize('ab')
assert not mini.recognize('aaaaa')
assert not mini.recognize('bbbbb')
assert not mini.recognize('abbabababa')

## Propuestas
- Implemente el resto de las operaciones entre autómatas vistas en conferencia:
    - Complemento
    - Intersección
    - Diferencia
    - Reverso
    - Zip